In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import re
import numpy as np
import matplotlib.pyplot as plt

In [2]:
discharge = pd.read_csv("discharge/discharge.csv", nrows=1000)

In [3]:
discharge.head()

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,\nName: ___ Unit No: _...
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,\nName: ___ Unit No: _...
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,\nName: ___ Unit No: _...
3,10000032-DS-24,10000032,25742920,DS,24,2180-08-07 00:00:00,2180-08-10 05:43:00,\nName: ___ Unit No: _...
4,10000084-DS-17,10000084,23052089,DS,17,2160-11-25 00:00:00,2160-11-25 15:09:00,\nName: ___ Unit No: __...


In [4]:
discharge.loc[10,'text']

' \nName:  ___.          Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nPatient recorded as having No Known Allergies to Drugs\n \nAttending: ___\n \nChief Complaint:\nAbdominal distention.\n \nMajor Surgical or Invasive Procedure:\nParacentesis.\n \nHistory of Present Illness:\nThis is a very nice ___ woman with ETOH abuse who \npresents with constipation, abdominal distention x 10 days.  \nPatient drinks about ___ glasses of wine per night and went on a \n2 month binge drinking ~10/day, which ended about 1 month ago.  \nShe noted abdominal distension progressive over the past week \nand has also not had a solid stool x 7 days.  She denies any \nf/C/NS, CP/SOB/DOE or decrease in her excercise tolerance.  No \nrecent travel but has traveled to ___ and ___ \npreviously.  No NSAIDs, Tylenol or OTC medications other than \noccasional peptobismol.  She notes that she has missed h

In [5]:
seed = 123
for statement in discharge.loc[0:3, 'text'].sample(3, random_state=seed):
    print(statement)
    print('-------------------------------------------------------------------------')

 
Name:  ___                     Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
Percocet / Vicodin
 
Attending: ___.
 
Chief Complaint:
Abdominal pain
 
Major Surgical or Invasive Procedure:
___ Paracentesis

 
History of Present Illness:
___ w/ HIV on HAART, COPD on 3L home O2 (though sat'ing fine on 
RA on admission), HCV cirrhosis c/b ascites requiring biweekly 
therapeutic paracenteses, hepatic encephalopathy; not on 
transplant list ___ comorbidities) p/w worsening girth and abd 
pain. She'd been having pain from her ascites, and felt overdue 
for a paracentesis. She last had paracentesis on the ___. 
She reported that she began feeling worsening abdominal pain 
(constant, epigastric and radiates to back, not a/w food) on 
___, which increased in severity throughout the course of the 
day. She was brought to the ED by her son because of this pain. 
She had no confusion and was al

In [6]:
radiology = pd.read_csv("radiology/radiology.csv", nrows=1000)

In [7]:
radiology.head()

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-RR-14,10000032,22595853.0,RR,14,2180-05-06 21:19:00,2180-05-06 23:32:00,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...
1,10000032-RR-15,10000032,22595853.0,RR,15,2180-05-06 23:00:00,2180-05-06 23:26:00,EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ...
2,10000032-RR-16,10000032,22595853.0,RR,16,2180-05-07 09:55:00,2180-05-07 11:15:00,"INDICATION: ___ HCV cirrhosis c/b ascites, hi..."
3,10000032-RR-18,10000032,NaN,RR,18,2180-06-03 12:46:00,2180-06-03 14:01:00,EXAMINATION: Ultrasound-guided paracentesis.\...
4,10000032-RR-20,10000032,NaN,RR,20,2180-07-08 13:18:00,2180-07-08 14:15:00,EXAMINATION: Paracentesis\n\nINDICATION: ___...


In [8]:
for statement in radiology.loc[0:3, 'text'].sample(3, random_state=seed):
    print(statement)
    print('-------------------------------------------------------------------------')

EXAMINATION:  Ultrasound-guided paracentesis.

INDICATION:  ___ year old woman with cirrhosis, ascites despite diuretics  //
Please do LVP with albumin post per protocol

TECHNIQUE:  Ultrasound guided therapeutic paracentesis

COMPARISON:  ___.

FINDINGS: 

Limited grayscale ultrasound imaging of the abdomen demonstrated
moderateascites. A suitable target in the deepest pocket in the right lower
quadrant was selected for paracentesis.

PROCEDURE:  The procedure, risks, benefits and alternatives were discussed
with the patient and written informed consent was obtained.

A preprocedure time-out was performed discussing the planned procedure,
confirming the patient's identity with 3 identifiers, and reviewing a
checklist per ___ protocol.

Under ultrasound guidance, an entrance site was selected and the skin was
prepped and draped in the usual sterile fashion. 1% lidocaine was instilled
for local anesthesia.

A 5 ___ catheter was advanced into the largest fluid pocket in the right
lower q

# we are not using dropna() while reading the files as it is taking out lot of rows.

core: admissions, patients, transfer. 
hos: 
icu:

In [10]:
#  pd.read_csv('data/hos/admissions.csv').dropna().reset_index(drop = True) #(5563, 16)


admissions = pd.read_csv('data/hos/admissions.csv').reset_index(drop = True) #431231 rows × 16 columns

In [11]:
admissions.head(10)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0
5,10000084,23052089,2160-11-21 01:56:00,2160-11-25 14:52:00,NaN,EW EMER.,P6957U,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,2160-11-20 20:36:00,2160-11-21 03:20:00,0
6,10000084,29888819,2160-12-28 05:11:00,2160-12-28 16:07:00,NaN,EU OBSERVATION,P63AD6,PHYSICIAN REFERRAL,NaN,Medicare,ENGLISH,MARRIED,WHITE,2160-12-27 18:32:00,2160-12-28 16:07:00,0
7,10000108,27250926,2163-09-27 23:17:00,2163-09-28 09:04:00,NaN,EU OBSERVATION,P38XXV,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2163-09-27 16:18:00,2163-09-28 09:04:00,0
8,10000117,22927623,2181-11-15 02:05:00,2181-11-15 14:52:00,NaN,EU OBSERVATION,P2358X,EMERGENCY ROOM,NaN,Other,ENGLISH,DIVORCED,WHITE,2181-11-14 21:51:00,2181-11-15 09:57:00,0
9,10000117,27988844,2183-09-18 18:10:00,2183-09-21 16:30:00,NaN,OBSERVATION ADMIT,P75S70,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Other,ENGLISH,DIVORCED,WHITE,2183-09-18 08:41:00,2183-09-18 20:20:00,0


In [12]:
# pd.read_csv('data/hos/patients.csv').dropna().reset_index(drop = True) #29076 rows × 6 columns

patients = pd.read_csv('data/hos/patients.csv').reset_index(drop = True)

#when using the dropna call we ony get 29076 row and without it we have 299712 rows.


In [13]:
patients.head(10)

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,NaN
5,10000108,M,25,2163,2014 - 2016,NaN
6,10000115,M,24,2154,2017 - 2019,NaN
7,10000117,F,48,2174,2008 - 2010,NaN
8,10000178,F,59,2157,2017 - 2019,NaN
9,10000248,M,34,2192,2014 - 2016,NaN


In [14]:
transfers = pd.read_csv('data/hos/transfers.csv').reset_index(drop = True) #1890972 rows × 7 columns

In [15]:
patients.shape #(5563, 16)

(299712, 6)

In [16]:
patient_diagnosis = pd.read_csv('data/hos/diagnoses_icd.csv').reset_index(drop = True)

In [17]:
patient_diagnosis.shape #(4756326, 5)

(4756326, 5)

In [18]:
pd.read_csv('data/hos/d_icd_diagnoses.csv').reset_index(drop = True)

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A
...,...,...,...
109770,Z992,10,Dependence on renal dialysis
109771,Z993,10,Dependence on wheelchair
109772,Z998,10,Dependence on other enabling machines and devices
109773,Z9981,10,Dependence on supplemental oxygen


In [19]:
pd.read_csv('data/hos/d_icd_procedures.csv').reset_index(drop = True)

,icd_code,icd_version,long_title
0,0001,9,Therapeutic ultrasound of vessels of head and ...
1,0002,9,Therapeutic ultrasound of heart
2,0003,9,Therapeutic ultrasound of peripheral vascular ...
3,0009,9,Other therapeutic ultrasound
4,001,10,"Central Nervous System and Cranial Nerves, Bypass"
...,...,...,...
85252,XW0DXV5,10,Introduction of Gilteritinib Antineoplastic in...
85253,XXE,10,"New Technology, Physiological Systems, Measure..."
85254,XXE5XM5,10,"Measurement of Infection, Whole Blood Nucleic ..."
85255,XY0,10,"New Technology, Extracorporeal, Introduction"


In [21]:
pd.read_csv('data/hos/emar.csv', nrows=100).reset_index(drop = True)

,subject_id,hadm_id,emar_id,emar_seq,poe_id,pharmacy_id,enter_provider_id,charttime,medication,event_txt,scheduletime,storetime
0,10000032,22595853,10000032-10,10,10000032-36,48770010.0,NaN,2180-05-07 00:44:00,Potassium Chloride,Administered,2180-05-07 00:44:00,2180-05-07 00:44:00
1,10000032,22595853,10000032-11,11,10000032-22,14779570.0,NaN,2180-05-07 00:44:00,Sodium Chloride 0.9% Flush,Flushed,2180-05-07 00:44:00,2180-05-07 00:44:00
2,10000032,22595853,10000032-12,12,10000032-37,93463122.0,NaN,2180-05-07 06:10:00,Ipratropium Bromide Neb,Administered,2180-05-07 06:00:00,2180-05-07 06:10:00
3,10000032,22595853,10000032-13,13,10000032-28,42497745.0,NaN,2180-05-07 05:00:00,Albuterol Inhaler,Administered,2180-05-07 06:29:00,2180-05-07 06:29:00
4,10000032,22595853,10000032-14,14,10000032-29,69131933.0,NaN,2180-05-07 07:51:00,Emtricitabine-Tenofovir (Truvada),Administered,2180-05-07 08:00:00,2180-05-07 07:56:00
...,...,...,...,...,...,...,...,...,...,...,...,...
95,10000032,29079034,10000032-56,56,10000032-125,NaN,NaN,2180-07-23 08:39:00,Insulin,Administered,2180-07-23 08:39:00,2180-07-23 08:42:00
96,10000032,29079034,10000032-57,57,10000032-128,92009906.0,NaN,2180-07-23 09:51:00,Albumin 5% (25g / 500mL),Administered,2180-07-23 09:51:00,2180-07-23 09:51:00
97,10000032,29079034,10000032-58,58,10000032-129,NaN,NaN,2180-07-23 10:16:00,Sodium Polystyrene Sulfonate,Hold Dose,2180-07-23 10:16:00,2180-07-23 10:16:00
98,10000032,29079034,10000032-59,59,10000032-120,NaN,NaN,2180-07-23 10:21:00,Lactulose,Administered,2180-07-23 11:00:00,2180-07-23 10:21:00


In [22]:
pd.read_csv('data/hos/emar_detail.csv', nrows=100).reset_index(drop = True)

,subject_id,emar_id,emar_seq,parent_field_ordinal,administration_type,pharmacy_id,barcode_type,reason_for_no_barcode,complete_dose_not_given,dose_due,...,infusion_rate_unit,route,infusion_complete,completion_interval,new_iv_bag_hung,continued_infusion_in_other_location,restart_interval,side,site,non_formulary_visual_verification
0,10000032,10000032-10,10,1.1,NaN,48770010.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000032,10000032-10,10,1.2,NaN,48770010.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000032,10000032-10,10,1.3,NaN,48770010.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000032,10000032-10,10,1.4,NaN,48770010.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000032,10000032-10,10,NaN,Standard Maintenance Medication,NaN,NaN,NaN,No,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10000032,10000032-142,142,NaN,Standard Maintenance Medication,NaN,NaN,NaN,No,30,...,NaN,PO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,10000032,10000032-143,143,1.1,NaN,82541937.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,10000032,10000032-143,143,NaN,Standard Maintenance Medication,NaN,NaN,NaN,No,400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,10000032,10000032-144,144,1.1,NaN,71423977.0,if,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
